# Проект. Анализ базы данных SQL.

#### Цель - проанализировать базу данных купиленного крупного сервиса для чтения книг по подписке.

Задачи:

* познакомиться с хранящимися в базе данными
* проанализировать результаты запросов.

Данные расположены в 5 таблицах, содержащих информацию о книгах, издательствах, авторах, пользовательских оценках и обзорах книг. Вместе с заданием были выданы реквезиты для подключения к БД и сертификат.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
pd.set_option('display.max_colwidth', None)

In [3]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 


**Выведем первые строки каждой из таблиц**

In [4]:
def slt(query):
    display(pd.io.sql.read_sql(query, con = engine))

In [5]:
books = ''' SELECT * FROM books LIMIT 5'''

In [6]:
slt(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [7]:
authors = ''' SELECT * FROM authors LIMIT 5'''

In [8]:
slt(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [9]:
publishers = ''' SELECT * FROM publishers LIMIT 5'''

In [10]:
slt(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [11]:
ratings = ''' SELECT * FROM ratings LIMIT 5'''

In [12]:
slt(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [13]:
reviews = ''' SELECT * FROM reviews LIMIT 5'''

In [14]:
slt(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


**Сделаем по одному SQL-запросу для решения каждого задания**

Задания:
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Сколько книг вышло после 1 января 2000 года**

In [15]:
ex1 = ''' SELECT COUNT(book_id) AS cnt_book
            FROM books
            WHERE publication_date > DATE('2000-01-01')
        '''

In [16]:
slt(ex1)

,cnt_book
0,819


После 1 января 2000 вышло 819 книг. (Если включая книги вышедшие 1 января (знак >=), будет 821)

**Для каждой книги посчитайте количество обзоров и среднюю оценку**

In [29]:
ex2 = ''' SELECT title, SUM(cnt_review) AS review_count, AVG(avg_raiting) AS average_raiting
            FROM books 
            LEFT JOIN (SELECT book_id, COUNT(review_id) AS cnt_review 
                        FROM reviews
                        GROUP BY book_id) AS sub
            ON books.book_id = sub.book_id
            LEFT JOIN (SELECT book_id, AVG(rating) AS avg_raiting
                        FROM ratings 
                        GROUP BY book_id) AS sub_2
            ON books.book_id = sub_2.book_id
            GROUP BY books.title
            
        '''

In [30]:
slt(ex2)

,title,review_count,average_raiting
0,The Count of Monte Cristo,5.0,4.217391
1,Count Zero (Sprawl #2),2.0,2.500000
2,The Botany of Desire: A Plant's-Eye View of the World,2.0,3.500000
3,The Poisonwood Bible,5.0,4.363636
4,The Canterbury Tales,3.0,3.333333
...,...,...,...
994,In the Heart of the Sea: The Tragedy of the Whaleship Essex,3.0,3.333333
995,Of Love and Other Demons,2.0,4.500000
996,Welcome to Temptation (Dempseys #1),2.0,5.000000
997,World's End (The Sandman #8),2.0,4.500000


Получили таблицу из 999 строк с названием книги, количеством обзоров и средней оценкой

**Определите издательство, которое выпустило наибольшее число книг толще 50 страниц**

In [19]:
ex3 = ''' SELECT publishers.publisher, COUNT(books.book_id) AS cnt
             FROM books 
             LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
             WHERE books.num_pages > 50
             GROUP BY publishers.publisher 
             ORDER BY cnt DESC
             LIMIT 1
        '''

In [20]:
slt(ex3)

,publisher,cnt
0,Penguin Books,42


Издательство Penguin Books выпустило 42 книги толще 50 страниц

**Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками**

In [21]:
ex4 = '''SELECT authors.author, ROUND(AVG(ratings.rating), 2)
         FROM books
         LEFT JOIN authors ON books.author_id = authors.author_id 
         LEFT JOIN ratings ON books.book_id = ratings.book_id
         WHERE books.book_id IN 
                  (SELECT book_id
                   FROM ratings 
                   GROUP BY book_id
                   HAVING COUNT(rating_id) > 50)
         GROUP BY authors.author
         ORDER BY AVG(ratings.rating) DESC
         LIMIT 1
     '''

In [22]:
slt(ex4)

,author,round
0,J.K. Rowling/Mary GrandPré,4.29


Самая выская средняя оценка книг у J.K. Rowling/Mary - 4.29

**Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок**

In [23]:
ex5 = ''' SELECT ROUND(AVG(cnt)) AS avg_cnt
          FROM (SELECT username, COUNT(review_id) as cnt
          FROM reviews
          WHERE username IN (SELECT username
                                 FROM ratings 
                                  GROUP BY username
                                  HAVING COUNT(rating_id) > 50)
         GROUP BY username) as subquery                        
      '''

In [24]:
slt(ex5)

,avg_cnt
0,24.0


Пользователи, которые поставили больше 50 оценок, делают в среднем 24 обзора